<a href="https://colab.research.google.com/github/krixik-ai/krixik-docs/blob/main/docs/system/pipeline_creation/create_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import json
from pathlib import Path

# preparación de demo - incuye instanciación de secretos, instalación de requerimientos, y definición de rutas
if os.getenv("COLAB_RELEASE_TAG"):
    # si estás usando este notebook en Google Colab, ingresa tus secretos acá
    MY_API_KEY = "TU_API_KEY_VA_AQUI"
    MY_API_URL = "TU_API_URL_VA_AQUI"

    # si estás usando este notebook en Google Colab, instala requerimientos y descarga los subdirectorios requeridos
    # instala el cliente Python de Krixik
    !pip install krixik

    # instala github-clone, que permite clonación fácil de los subdirectorios del repositorio de documentación https://github.com/krixik-ai/krixik-docs
    !pip install github-clone

    # clona los conjuntos de datos
    if not Path("data").is_dir():
        !ghclone https://github.com/krixik-ai/krixik-docs/tree/es-main/data
    else:
        print("ya se clonaron los conjuntos de datos de documentación!")

    # define la variable 'data_dir' para tus rutas
    data_dir = "./data/"

    # crea directorio de salidas
    from pathlib import Path

    Path(data_dir + "/salidas").mkdir(parents=True, exist_ok=True)

else:
    # si estás usando una descarga local de la documentación, define las rutas relativas a la estructura local de la documentación
    # importa utilidades
    sys.path.append("../../../")

    # define la variable 'data_dir' para tus rutas
    data_dir = "../../../data/"

    # si estás usando este notebook localmente desde el repositorio de documentación Krixik, carga tus secretos de un archivo .env ubicado en la base del repositorio de documentación
    from dotenv import load_dotenv

    load_dotenv("../../../.env")

    MY_API_KEY = os.getenv("MY_API_KEY")
    MY_API_URL = os.getenv("MY_API_URL")


# importa Krixik e inicializa sesión con tus secretos personales
from krixik import krixik

krixik.init(api_key=MY_API_KEY, api_url=MY_API_URL)

## Creación de Pipelines
[🇺🇸 English version of this document](https://krixik-docs.readthedocs.io/latest/system/pipeline_creation/create_pipeline/)

Esta introducción a cómo crear pipelines se divide en las siguientes secciones:

- [El Método `create_pipeline`](#el-metodo-create_pipeline)
- [Un Pipeline de Módulo Único](#un-pipeline-de-modulo-unico)
- [Un Pipeline Multimodular](#un-pipeline-multimodular)
- [Validación de Secuencias de Módulos](#validacion-de-secuencias-de-modulos)
- [Repetición de Nombres de Pipelines](#repeticion-de-nombres-de-pipelines)

### El Metodo `create_pipeline`

El método `create_pipeline` instancia nuevos *pipelines*. Es un método muy simple que toma dos argumentos, y ambos son requeridos:

- `name` (str): El nombre de tu nuevo *pipeline*. Piénsalo bien, pues los nombres de los *pipelines* son sus identificadores únicos, así que dos *pipelines* no pueden compartir el mismo nombre.
- `module_chain` (list): La lista secuencial de módulos que forman tu nuevo pipeline.

Haz [clic aquí](../../modulos/introduccion_modulos.md) para ver la lista actual de módulos Krixik disponibles. Recuerda que, siempre y cuando sus entradas y salidas cuadren, cualquier combinación de módulos es válida, incluyendo aquellas en las que se repiten módulos.

### Un *Pipeline* de Modulo Unico

Usa el método `create_pipeline` para crear un *pipeline* de módulo único. Usarás el módulo [`parser`](../../modulos/modulos_de_funciones_de_apoyo/modulo_parser_fragmentacion.md), que fragmenta archivos de texto y los convierte en una serie de frases más cortas.

In [2]:
# crea un pipeline con un solo módulo parser
pipeline = krixik.create_pipeline(name="create_pipeline_1_parser", module_chain=["parser"])

Asegúrate que has [inicializado tu sesión](../inicializacion/inicializacion_y_autenticacion.md  ) antes de ejecutar este código.

Ten en cuenta que el argumento `name` puede ser el *string* que desees. Sin embargo, la lista `module_chain` solo puede estar compuesta de [identificadores de módulo](../metodos_de_conveniencia/metodos_de_conveniencia.md#ve-todos-los-modulos-disponibles-con-la-propiedad-available_modules) establecidos.

### Un *Pipeline* Multimodular

Ahora monta un *pipeline* que consiste de una secuencia de tres módulos: un módulo [`parser` (fragmentación de texto)](../../modulos/modulos_de_funciones_de_apoyo/modulo_parser_fragmentacion.md), un módulo [`text-embedder` (encaje léxico)](../../modulos/modulos_ia/modulo_text-embedder_encaje_lexico.md), y un módulo [`vector-db` (base de datos vectorial)](../../modulos/modulos_de_bases_de_datos/modulo_vector-db_base_de_datos_vectorial.md). Este `module_chain` surge a menudo, pues es la fórmula para el [pipeline básico de búsqueda semántica](../../ejemplos/ejemplos_pipelines_de_busqueda/multi_busqueda_semantica_basica.md) (también conocida como búsqueda vectorial).

Como puedes ver, la sintaxis para el montaje del *pipeline* es igual que antes. El orden de los módulos en el argumento `module_chain` es el orden secuencial en el que procesarán entradas al *pipeline*:

In [3]:
# crea un pipeline multimodular básico de búsqueda semántica
pipeline = krixik.create_pipeline(name="create_pipeline_2_parser_embedder_vector", module_chain=["parser", "text-embedder", "vector-db"])

Explora una variedad de ejemplos de *pipelines* multimodulares [aquí](../../ejemplos/introduccion_ejemplos_de_pipelines.md).

### Validacion de Secuencias de Modulos

Al ejecutar `create_pipeline` el cliente Krixik confirma que los módulos indicados funcionarán correctamente en la secuencia solicitada. Si no pueden—lo cual tiende a ser consecuencia de que la salida de un módulo no cuadra con la entrada del siguiente módulo—se lanza una excepción local explicativa.

Por ejemplo, tratar de crear un *pipeline* bimodular que consiste de un módulo [`parser` (fragmentación de texto)](../../modulos/modulos_de_funciones_de_apoyo/modulo_parser_fragmentacion.md) y un módulo [`caption` (leyenda de imagen)](../../modulos/modulos_ia/modulo_caption_leyenda_de_imagen.md), en ese orden, fallará (como debe) y producirá una excepción local. Esto es porque el módulo [`parser`](../../modulos/modulos_de_funciones_de_apoyo/modulo_parser_fragmentacion.md) devuelve un archivo JSON como salida, mientras que el módulo [`caption`](../../modulos/modulos_ia/modulo_caption_leyenda_de_imagen.md) solo toma imágenes como entrada, como indica el mensaje que puedes ver en seguida:

In [4]:
# intenta crear un pipeline compuesto de un módulo parser seguido de un módulo caption
pipeline = krixik.create_pipeline(name="create_pipeline_3_parser_caption", module_chain=["parser", "caption"])

TypeError: format type mismatch between parser - whose output format is json - and caption - whose input format is image

### Repeticion de Nombres de Pipelines

Krixik no te permite crear un *pipeline* con el `name` de otro *pipeline* que ya has creado. La única excepción es si el nuevo *pipeline* tiene una cadena de módulos idéntica a la anterior.

Ten en cuenta que si intentas crear un nuevo *pipeline* con el `name` de un *pipeline* anterior y con un `module_chain` diferente, la creación inicial del *pipeline* no fallará. Podrás ejecutar el método `create_pipeline` sin problema. Sin embargo, cuando dos *pipelines* con el mismo nombre y diferentes `module_chain`s existen y ya has [`procesado`](../parametros_y_procesar_archivos_a_traves_de_pipelines/metodo_process_procesar.md) al menos un archivo a través de uno de ellos, **no** podrás procesar un archivo a través del otro *pipeline* como consecuencia de esta duplicación de `name`s.

In [6]:
# elimina todos los datos procesados pertenecientes a este pipeline
krixik.reset_pipeline(pipeline)